In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import sklearn
import collections
from sklearn.model_selection import train_test_split
import json
import pylab
import tensorflow as tf
import tensorflow.math as tfmath
import tensorflow.keras as keras
from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as sk
import qkeras
from qkeras.qlayers import QDense, QActivation
from qkeras.qnormalization import QBatchNormalization
from qkeras.quantizers import quantized_bits, quantized_relu

2024-04-01 23:10:00.113652: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-01 23:10:00.480552: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
f=h5py.File('/eos/home-w/wsherman/AD_Work/n_tuples/40MHZ_data/background_for_training.h5','r')
Dataset=np.array(f["Particles"])

#for i, batch in enumerate(Dataset):
#  pt_sum=0
#  for j, particle in enumerate(Dataset[i,:,:]):
#    if particle[3]!=0:
#      pt_sum+=particle[0]
#  for j, particle in enumerate(Dataset[i,:,:]):
#    particle[0]=particle[0]/pt_sum
    
    
Data_Train=Dataset[0:2000000,:,0:3]
Data_Test=Dataset[2000001:3600000,:,0:3]
Data_Validate=Dataset[3600001:4000000,:,0:3]

Data_Train_Flat=np.reshape(Data_Train,(-1,57))
Data_Val_Flat=np.reshape(Data_Validate,(-1,57))

In [9]:
class Sampling(keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


def make_Qencoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(input_dim))

    x=QDense(h_dim_1,kernel_quantizer=quantized_bits(8),bias_quantizer=quantized_bits(8))(inputs)
    x=QActivation("quantized_relu(8)")(x)
    x=QDense(h_dim_2,kernel_quantizer=quantized_bits(8),bias_quantizer=quantized_bits(8))(x)
    x=QActivation("quantized_relu(8)")(x)
    z_mean=QDense(latent_dim, kernel_quantizer=quantized_bits(8),bias_quantizer=quantized_bits(8))(x)
    z_logvar=QDense(latent_dim, kernel_quantizer=quantized_bits(8),bias_quantizer=quantized_bits(8))(x)
    z=Sampling()([z_mean,z_logvar])
    encoder=keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')
    return encoder




def make_Qdecoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(latent_dim))
    x=QDense(h_dim_2, kernel_quantizer=quantized_bits(16),bias_quantizer=quantized_bits(16))(inputs)
    x=QActivation("quantized_relu(16)")(x)
    x=QDense(h_dim_1,kernel_quantizer=quantized_bits(16),bias_quantizer=quantized_bits(16))(x)
    x=QActivation("quantized_relu(16)")(x)
    z=QDense(input_dim,kernel_quantizer=quantized_bits(16),bias_quantizer=quantized_bits(16))(x)
    decoder=keras.Model(inputs,z,name='decoder')
    return decoder


def make_Qencoder_norm(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(input_dim))
    x=QBatchNormalization()(inputs)
    x=QDense(h_dim_1,kernel_quantizer=quantized_bits(16),bias_quantizer=quantized_bits(16))(x)
    x=QActivation("quantized_relu(16)")(x)
    x=QDense(h_dim_2,kernel_quantizer=quantized_bits(16),bias_quantizer=quantized_bits(16))(x)
    x=QActivation("quantized_relu(16)")(x)
    z_mean=QDense(latent_dim, kernel_quantizer=quantized_bits(16),bias_quantizer=quantized_bits(16))(x)
    z_logvar=QDense(latent_dim, kernel_quantizer=quantized_bits(16),bias_quantizer=quantized_bits(16))(x)
    z=Sampling()([z_mean,z_logvar])
    encoder=keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')
    return encoder



class QVAE_Model(keras.Model):
    def __init__(self,encoder,decoder,**kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.beta=1
        self.chi=1

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def set_beta(self,beta):
        self.beta=beta
    
    def set_chi(self,chi):
        self.chi=chi
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #making a masked loss function
            mask = K.cast(K.not_equal(data, 0), K.floatx())
            
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
            reconstruction_loss *=(1-self.beta)

            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *=self.beta
            kl_loss *=self.chi


            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reco_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        
        reconstruction = self.decoder(z)
        mask = K.cast(K.not_equal(data, 0), K.floatx())
        reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
        
        reconstruction_loss*=(1-self.beta)

        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        
        #KL loss changed abck to sum as in paper
        kl_loss = tf.reduce_sum(kl_loss)
        
        kl_loss *=self.beta
        
        total_loss = reconstruction_loss + kl_loss
        return {
            "loss": total_loss,
            "reco_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

    def call(self, data):
        z_mean,z_log_var,x = self.encoder(data)
        reconstruction = self.decoder(x)
        return {
            "z_mean": z_mean,
            "z_log_var": z_log_var,
            "reconstruction": reconstruction
        }

        
    

In [17]:
#Here is a normalized model with (1-beta)rl beta*Kl loss
beta=0.83
chi=10**8
vae_enc=make_Qencoder_norm(57,32,16,3)
vae_dec=make_Qdecoder(57,32,16,3)
vae_40MHZ=QVAE_Model(vae_enc,vae_dec)
vae_40MHZ.set_beta(beta)
vae_40MHZ.set_chi(chi)
opt=keras.optimizers.Adam(learning_rate=0.001)
vae_40MHZ.compile(optimizer=opt)


STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 100
BATCH_SIZE = 100

history = vae_40MHZ.fit(x=Data_Train_Flat, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True)
vae_40MHZ.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/Different_VAE_Models/Q_keras_16_bit_batchnormed_beta_{}_chi_{}_v4/'.format(beta,chi),save_format='tf')

Epoch 1/100
18000/18000 [==============================] - 85s 4ms/step - loss: 125359.5389 - reco_loss: 524.2701 - kl_loss: 40145.9453 - val_loss: 360.0016 - val_reco_loss: 359.9951 - val_kl_loss: 0.0066 - lr: 0.0010
Epoch 2/100
18000/18000 [==============================] - 73s 4ms/step - loss: 26105.0072 - reco_loss: 426.2299 - kl_loss: 25723.9414 - val_loss: 359.8913 - val_reco_loss: 359.8907 - val_kl_loss: 6.3571e-04 - lr: 0.0010
Epoch 3/100
18000/18000 [==============================] - 72s 4ms/step - loss: 26290.1770 - reco_loss: 427.1804 - kl_loss: 25754.4863 - val_loss: 359.9524 - val_reco_loss: 359.9464 - val_kl_loss: 0.0060 - lr: 0.0010
Epoch 4/100
18000/18000 [==============================] - 72s 4ms/step - loss: 26156.6472 - reco_loss: 424.9658 - kl_loss: 25754.1230 - val_loss: 359.7445 - val_reco_loss: 359.7415 - val_kl_loss: 0.0029 - lr: 0.0010
Epoch 5/100
18000/18000 [==============================] - 73s 4ms/step - loss: 26069.1941 - reco_loss: 432.7104 - kl_loss: 257